In [45]:
!pip install lizard

     |████████████████████████████████| 66 kB 1.2 MB/s eta 0:00:01


In [1]:
import pandas as pd
import numpy as np
import json
import hashlib
import os
import lizard

In [8]:
from git import Repo
repo_url = "https://github.com/"
dir_to_save = "/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/"
Repo.clone_from(os.path.join(repo_url, ), os.path.join(dir_to_save, ))

<git.repo.base.Repo '/home/amirmahdi/Desktop/a/.git'>

### Files name and their hash value
Making a list of files name and assigning a hash value to them. This hash value is going to be used as ID for each microbenchmark:

In [2]:
hash_dict = {}
filesname = []
df = pd.DataFrame(columns=["ID", "itr_to_stds_avrg", "itr_to_stds_std"])

In [3]:
directory = "/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/changepoints"

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    
    if os.path.isfile(f):
        hash_dict[ hashlib.sha1(str.encode(filename)).hexdigest() ] = filename 
        filesname.append(filename)

### Changepoints
Finfing the last change point of each of ten forks in each benchmark and computing the average and standard deviation of them:

In [4]:
directory = "/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/changepoints"

for filename in filesname:                     #For loop over each json file of changepoints
    f = os.path.join(directory, filename)
    
    if os.path.isfile(f):
        j_file = open(f)
        
        data = json.load(j_file)
        last_changepoints = []                 # it should be empty for each seperate benchmark...
        number_of_change_points=[]
        
        for fork_data in data:   #For loop over forks in the json file and capturing last change point of each fork
            number_of_change_points.append(len(fork_data)-1)
            
            if len(fork_data)==1:
                last_changepoints.append(0)     #if the fork lenght is 1, there is no change point
            else:
                last_changepoints.append(fork_data[-2])
        
        last_changepoint_avrg = np.mean(last_changepoints)  #computing average
        last_changepoint_std = np.std(last_changepoints)    #computing standard deviation
        number_of_change_points_avrg = np.mean(number_of_change_points)
        
        ID = hashlib.sha1(str.encode(filename)).hexdigest()                    #computing the ID of the benchmark
        new_row = pd.Series({"ID":ID, "itr_to_stds_avrg":last_changepoint_avrg, 
                                "itr_to_stds_std":last_changepoint_std, 
                             "Num_of_change_points_avrg":number_of_change_points_avrg})  #creating new row of data
        df = pd.concat([df, new_row.to_frame().T], ignore_index=True)          #adding the data to the dataframe

In [5]:
df

,ID,itr_to_stds_avrg,itr_to_stds_std,Num_of_change_points_avrg
0,91f071f1f69bf48297887879cf5e5613552a29bc,1740.1,1021.197968,3.1
1,fb2459909444ecbdf72e47a11179359e56778dcd,196.6,326.93247,1.3
2,1c51969ca9fc7c1b5a7ac533bfa76899ac30ed88,363.5,396.638689,1.8
3,2b3a09d375526c8e9c7641660de34577306fb84e,2256.7,982.812602,7.5
4,ca5d32819509b08c929db359c6f8dddad84e62f4,1628.1,734.577491,3.5
...,...,...,...,...
581,1a3b2d7b3a6ece03ec9db59ba9385a7199cace1b,1124.9,1081.148783,2.7
582,0b0eb41a2e416dd93cf156632edab78e7227fab6,1833.0,913.468883,5.3
583,8ac7776547858abdceb1e0b055d762870adcc23a,582.0,0.0,3.0
584,ce0f7ccdb51b4a6800adab9c0668907d4665401d,1113.2,824.752423,1.9


In [45]:
df = df.set_index("ID")

### Timeseries
Getting timeseries of each benchmarks which contains 10 forks. Then we merge all o the 10 forks into one by computing the average of them. Now having the average, we compute the average and standard deviation of the result, which represents the average and standard deviation of the benchmark's timeseries:

In [46]:
directory = "/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/timeseries"

for filename in filesname:
    f = os.path.join(directory, filename)
    
    if os.path.isfile(f):
        j_file = open(f)
        timeseries_data = json.load(j_file)         # Loading the benchmark's timeseries which is in json
        
        timeseries_df = pd.DataFrame(timeseries_data) # changing json into Pandas Dataframe
        bench_timeseries__forks_avrg_df = timeseries_df.mean() # computing the mean of forks
        bench_timeseries_std = bench_timeseries__forks_avrg_df.std() 
        bench_timeseries_avrg = bench_timeseries__forks_avrg_df.mean()
        
        ID = hashlib.sha1(str.encode(filename)).hexdigest()
        
        df.at[ID, "STD"] = bench_timeseries_std
        df.at[ID, "AVRG"] = bench_timeseries_avrg

In [16]:
df

,itr_to_stds_avrg,itr_to_stds_std,Num_of_change_points_avrg,STD,AVRG
ID,,,,,
91f071f1f69bf48297887879cf5e5613552a29bc,1740.1,1021.197968,3.1,6.510561e-08,1.869715e-07
fb2459909444ecbdf72e47a11179359e56778dcd,196.6,326.93247,1.3,6.980091e-09,1.831741e-07
1c51969ca9fc7c1b5a7ac533bfa76899ac30ed88,363.5,396.638689,1.8,1.043777e-08,8.916153e-08
2b3a09d375526c8e9c7641660de34577306fb84e,2256.7,982.812602,7.5,2.670392e-07,6.533268e-07
ca5d32819509b08c929db359c6f8dddad84e62f4,1628.1,734.577491,3.5,1.180394e-04,1.245521e-03
...,...,...,...,...,...
1a3b2d7b3a6ece03ec9db59ba9385a7199cace1b,1124.9,1081.148783,2.7,1.093954e-02,5.990179e-04
0b0eb41a2e416dd93cf156632edab78e7227fab6,1833.0,913.468883,5.3,1.698672e-07,2.915286e-07
8ac7776547858abdceb1e0b055d762870adcc23a,582.0,0.0,3.0,1.060187e-02,3.083067e-02


### Static features by using Lizard framework:
In this part, first we need to find the benchmark file related to the benchmark under study among all existing projects, so that we could work on static features of that file then.

To do this we are using the Filesname we have which are in format below:

 github_organization __ github_project # benchmark_method # benchmark_parameters.json
 
Using the filesname, we extract the project name, benchmark's file name and the method under study. By having these informations and the method "find_files", we get to the benchmarks's file containing source code.

Having access to the source code, now we use Lizard framework to extract following static features from source code:

* Bench_File_Nloc (Benchmark's number of lines of code)
* Bench_Method_Nloc (Benchmark's method under study number of lines of code)
* Bench_Method_NestingLevel (Nesting level of the method under study)
* Bench_Method_CCN (Cyclomatic Complexity Number of the method under study)
* Bench_File_CCN_Avrg (Cyclomatic Complexity Number of the whole benchmark's file)

We also include some other information:

* Project_Name
* Bench_File_Name
* Bench_Method_Name

In [5]:
def find_files(filename, search_path):  # Searching for the given filename in given path.
    result = []
    #print(filename)
    for root, dirs, files in os.walk(search_path):
        #print(root)
        if filename in files:
            result.append(os.path.join(root, filename))
            result.append(root) # adding the directory without including the file's name in it
    return result

In [69]:
search_path_base = "/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects"
r=[]
for filename in filesname:    # Looping over all files to find the related benchmark to extract its features
    #print(filename)
    benchmark_name = filename.split("#")[1].split(".")[-2]+".java"
    benchmark_method = filename.split("#")[1].split(".")[-1] 
    project_name = filename.split("#")[0]
    #print(project_name)
    search_path = os.path.join(search_path_base, project_name) #Creating the project's directory which we should search to find the benchmark's file
    #print(search_path)
    try:
        benchmark_dir = find_files(benchmark_name, search_path)[0] # Searching for the benchmark's file in the project directory related to the benchmark under study
    except IndexError:
        # Printing error if the file couldn't be found, and then skip to the next file (benchmark)
        """print(f"File: {benchmark_name} NOT FOUND (Benchmark: {filename})")"""
        
        benchmark_name = filename.split("#")[1].split(".")[-3]+".java"
        benchmark_method = filename.split("#")[1].split(".")[-1] 
        project_name = filename.split("#")[0]
        #print(project_name)
        search_path = os.path.join(search_path_base, project_name)
        benchmark_dir = find_files(benchmark_name, search_path)[0]
        
        Bench_anlysis = lizard.analyze_file(benchmark_dir) # Passing the benchmark's file to the Lizard to do its static analysis
        bench_file_nloc = list(Bench_anlysis.__dict__.values())[1] 
        bench_file_ccn = []
        for j in range(len(Bench_anlysis.function_list)): # Looping over all functions in the benchmark to find the benchmark method under study
            bench_file_ccn.append(dict(Bench_anlysis.function_list[j].__dict__)['cyclomatic_complexity']) # saving the CCN value of each function to compute the average CCN of the whole benchmark
            if dict(Bench_anlysis.function_list[j].__dict__)["name"].split("::")[1] == benchmark_method: # Checking if the current function is the function under study or not
                bench_nloc = dict(Bench_anlysis.function_list[j].__dict__)["nloc"] 
                bench_ccn = dict(Bench_anlysis.function_list[j].__dict__)['cyclomatic_complexity'] 
                bench_nesting_level = dict(Bench_anlysis.function_list[j].__dict__)['top_nesting_level'] 
        bench_file_cnn_avrg = np.mean(bench_file_ccn) #
        
        ID = hashlib.sha1(str.encode(filename)).hexdigest()
        
        df.at[ID, "Project_Name"] = project_name
        df.at[ID, "Bench_File_Name"] = benchmark_name
        df.at[ID, "Bench_Method_Name"] = benchmark_method
        df.at[ID, "Bench_File_Nloc"] = bench_file_nloc
        df.at[ID, "Bench_Method_Nloc"] = bench_nloc
        df.at[ID, "Bench_Method_NestingLevel"] = bench_nesting_level
        df.at[ID, "Bench_Method_CCN"] = bench_ccn
        df.at[ID, "Bench_File_CCN_Avrg"] = bench_file_cnn_avrg
        continue
    else:
        Bench_anlysis = lizard.analyze_file(benchmark_dir) # Passing the benchmark's file to the Lizard to do its static analysis
        bench_file_nloc = list(Bench_anlysis.__dict__.values())[1] 
        bench_file_ccn = []
        for j in range(len(Bench_anlysis.function_list)): # Looping over all functions in the benchmark to find the benchmark method under study
            bench_file_ccn.append(dict(Bench_anlysis.function_list[j].__dict__)['cyclomatic_complexity']) # saving the CCN value of each function to compute the average CCN of the whole benchmark
            if dict(Bench_anlysis.function_list[j].__dict__)["name"].split("::")[1] == benchmark_method: # Checking if the current function is the function under study or not
                bench_nloc = dict(Bench_anlysis.function_list[j].__dict__)["nloc"] 
                bench_ccn = dict(Bench_anlysis.function_list[j].__dict__)['cyclomatic_complexity'] 
                bench_nesting_level = dict(Bench_anlysis.function_list[j].__dict__)['top_nesting_level'] 
        bench_file_cnn_avrg = np.mean(bench_file_ccn) #
        
        ID = hashlib.sha1(str.encode(filename)).hexdigest()
        
        df.at[ID, "Project_Name"] = project_name
        df.at[ID, "Bench_File_Name"] = benchmark_name
        df.at[ID, "Bench_Method_Name"] = benchmark_method
        df.at[ID, "Bench_File_Nloc"] = bench_file_nloc
        df.at[ID, "Bench_Method_Nloc"] = bench_nloc
        df.at[ID, "Bench_Method_NestingLevel"] = bench_nesting_level
        df.at[ID, "Bench_Method_CCN"] = bench_ccn
        df.at[ID, "Bench_File_CCN_Avrg"] = bench_file_cnn_avrg

In [ ]:
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/jgrapht__jgrapht/jgrapht-jgrapht-1.5.0/jgrapht-core/src/test/java/org/jgrapht/perf/graph/GraphPerformanceTest.java
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/jgrapht__jgrapht/jgrapht-jgrapht-1.5.0/jgrapht-core/src/test/java/org/jgrapht/perf/clique/MaximalCliqueEnumerationPerformanceTest.java


### Static features by using srcML framework:
Regarding to finding and dealing with benchmark files, this part is doing the same as last part using Lizard framework. However, there were a few files that did not follow our assumption about the benchmarks name (the file name was third one from end instead of second one) which we have dealt with it in expect part. This part is also added to the last part using Lizard, as we had same problem there as well.

Using srcML let us to extract some other static features which Lizard could not provide. These features are as follows:

* Number_of_tryCatch 
* Number_of_switchCase
* Number_of_systemCalls
* Synchronization
* Number_of_forLoops
* Number_of_whileLoops
* Number_of_packages
* Number_of_functionCalls

We also include some other features using files name (without using srcML):

* Number_of_parameters
* Benchmark_name_length

In [27]:
search_path_base = "/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects"
r=[]
for filename in filesname:    # Looping over all files to find the related benchmark to extract its features
    #print(filename)
    benchmark_name = filename.split("#")[1].split(".")[-2]+".java"
    benchmark_method = filename.split("#")[1].split(".")[-1] 
    benchmark_method = benchmark_method.split(" ")[0]
    project_name = filename.split("#")[0]
    potential_parameters = filename.split("#")[-1].split("&")
    number_of_parameters = len(potential_parameters) if potential_parameters[0] != ".json" else 0
    #print(project_name)
    search_path = os.path.join(search_path_base, project_name) #Creating the project's directory which we should search to find the benchmark's file
    #print(search_path)
    try:
        benchmark_dir = find_files(benchmark_name, search_path) # Searching for the benchmark's file in the project directory related to the benchmark under study
        benchmark_path = benchmark_dir[1] # benchmark_path is the benchmark directory without the file's name included in it.
    except IndexError:
        # Printing error if the file couldn't be found, and then skip to the next file (benchmark)
        """print(f"File: {benchmark_name} NOT FOUND (Project Name: {project_name})")"""
        
        benchmark_name = filename.split("#")[1].split(".")[-3]+".java"
        benchmark_method = filename.split("#")[1].split(".")[-1] 
        project_name = filename.split("#")[0]
        #print(project_name)
        search_path = os.path.join(search_path_base, project_name)
        benchmark_dir = find_files(benchmark_name, search_path)[0]
        benchmark_path = benchmark_dir[1]
        
        %cd {benchmark_path}
        xml_file_name = project_name + ".xml"
        !srcml {benchmark_name} -o {xml_file_name}
        #print(benchmark_name)
        #print(benchmark_method)
        
        """xpath_command_for_parameter = f"//src:function[src:name/text()={benchmark_method}]/src:parameter_list//src:parameter"
        number_of_parameters = !srcml --xpath "count({xpath_command_for_parameter})" {xml_file_name}
        #print(number_of_parameters)
        #break
        #print(number_of_parameters)"""
        xpath_command_for_try_catch = f"//src:function[src:name/text()='{benchmark_method}']//src:try"
        #xpath_command_for_try_catch = "//src:function[src:name/text()='runtime_sparse_serialize_10_int_fields']//src:try"
        number_of_try_catch = !srcml --xpath "count({xpath_command_for_try_catch})" {xml_file_name}
        xpath_command_for_switch = f"//src:function[src:name/text()='{benchmark_method}']//src:switch"
        number_of_switch = !srcml --xpath "count({xpath_command_for_switch})" {xml_file_name}
        xpath_command_for_systemCalls = f"//src:function[src:name/text()='{benchmark_method}']//src:call/src:name[src:name/text()='System']"
        number_of_systemCalls = !srcml --xpath "count({xpath_command_for_systemCalls})" {xml_file_name}
        xpath_command_for_threads = f"//src:function[src:name/text()='{benchmark_method}']//src:call/src:name[src:name/text()='Thread']"
        exist_thread = !srcml --xpath "count({xpath_command_for_threads})" {xml_file_name}
        xpath_command_for_synchronized = f"//src:function[src:name/text()='{benchmark_method}']//src:synchronized" 
        exist_synchronized = !srcml --xpath "count({xpath_command_for_synchronized})" {xml_file_name}
        xpath_command_for_forLoops = f"//src:function[src:name/text()='{benchmark_method}']//src:for"
        number_of_forLoops = !srcml --xpath "count({xpath_command_for_forLoops})" {xml_file_name}
        xpath_command_for_whileLoops = f"//src:function[src:name/text()='{benchmark_method}']//src:while"
        number_of_whileLoops = !srcml --xpath "count({xpath_command_for_whileLoops})" {xml_file_name}
        xpath_command_for_packages = "//src:import"
        number_of_packages = !srcml --xpath "count({xpath_command_for_packages})" {xml_file_name}
        xpath_command_for_functionCalls = f"//src:function[src:name/text()='{benchmark_method}']//src:call"
        number_of_functionCalls = !srcml --xpath "count({xpath_command_for_functionCalls})" {xml_file_name}
        
        ID = hashlib.sha1(str.encode(filename)).hexdigest()
        
        #df.at[ID, "Number_of_parameters"] = number_of_parameters[0] #alternative
        df.at[ID, "Number_of_tryCatch"] = number_of_try_catch[0] #
        df.at[ID, "Number_of_switchCase"] = number_of_switch[0] #
        df.at[ID, "Number_of_systemCalls"] = number_of_systemCalls[0] #
        df.at[ID, "Synchronization"] = 1 if exist_thread[0] != '0' and exist_synchronized[0] != "0" else 0
        df.at[ID, "Number_of_forLoops"] = number_of_forLoops[0] #
        df.at[ID, "Number_of_whileLoops"] = number_of_whileLoops[0] #
        df.at[ID, "Number_of_packages"] = number_of_packages[0] #
        df.at[ID, "Number_of_functionCalls"] = number_of_functionCalls[0] #
        df.at[ID, "Number_of_parameters"] = number_of_parameters
        df.at[ID, "Benchmark_name_length"] = len(benchmark_name)
        
        continue
    else:
        
        %cd {benchmark_path}
        xml_file_name = project_name + ".xml"
        !srcml {benchmark_name} -o {xml_file_name}
        #print(benchmark_name)
        #print(benchmark_method)
        
        """xpath_command_for_parameter = f"//src:function[src:name/text()={benchmark_method}]/src:parameter_list//src:parameter"
        number_of_parameters = !srcml --xpath "count({xpath_command_for_parameter})" {xml_file_name}
        #print(number_of_parameters)
        #break
        #print(number_of_parameters)"""
        xpath_command_for_try_catch = f"//src:function[src:name/text()='{benchmark_method}']//src:try"
        #xpath_command_for_try_catch = "//src:function[src:name/text()='runtime_sparse_serialize_10_int_fields']//src:try"
        number_of_try_catch = !srcml --xpath "count({xpath_command_for_try_catch})" {xml_file_name}
        xpath_command_for_switch = f"//src:function[src:name/text()='{benchmark_method}']//src:switch"
        number_of_switch = !srcml --xpath "count({xpath_command_for_switch})" {xml_file_name}
        xpath_command_for_systemCalls = f"//src:function[src:name/text()='{benchmark_method}']//src:call/src:name[src:name/text()='System']"
        number_of_systemCalls = !srcml --xpath "count({xpath_command_for_systemCalls})" {xml_file_name}
        xpath_command_for_threads = f"//src:function[src:name/text()='{benchmark_method}']//src:call/src:name[src:name/text()='Thread']"
        exist_thread = !srcml --xpath "count({xpath_command_for_threads})" {xml_file_name}
        xpath_command_for_synchronized = f"//src:function[src:name/text()='{benchmark_method}']//src:synchronized" 
        exist_synchronized = !srcml --xpath "count({xpath_command_for_synchronized})" {xml_file_name}
        xpath_command_for_forLoops = f"//src:function[src:name/text()='{benchmark_method}']//src:for"
        number_of_forLoops = !srcml --xpath "count({xpath_command_for_forLoops})" {xml_file_name}
        xpath_command_for_whileLoops = f"//src:function[src:name/text()='{benchmark_method}']//src:while"
        number_of_whileLoops = !srcml --xpath "count({xpath_command_for_whileLoops})" {xml_file_name}
        xpath_command_for_packages = "//src:import"
        number_of_packages = !srcml --xpath "count({xpath_command_for_packages})" {xml_file_name}
        xpath_command_for_functionCalls = f"//src:function[src:name/text()='{benchmark_method}']//src:call"
        number_of_functionCalls = !srcml --xpath "count({xpath_command_for_functionCalls})" {xml_file_name}
        
        ID = hashlib.sha1(str.encode(filename)).hexdigest()
        
        #df.at[ID, "Number_of_parameters"] = number_of_parameters[0] #alternative
        df.at[ID, "Number_of_tryCatch"] = number_of_try_catch[0] #
        df.at[ID, "Number_of_switchCase"] = number_of_switch[0] #
        df.at[ID, "Number_of_systemCalls"] = number_of_systemCalls[0] #
        df.at[ID, "Synchronization"] = 1 if exist_thread[0] != '0' and exist_synchronized[0] != "0" else 0
        df.at[ID, "Number_of_forLoops"] = number_of_forLoops[0] #
        df.at[ID, "Number_of_whileLoops"] = number_of_whileLoops[0] #
        df.at[ID, "Number_of_packages"] = number_of_packages[0] #
        df.at[ID, "Number_of_functionCalls"] = number_of_functionCalls[0] #
        df.at[ID, "Number_of_parameters"] = number_of_parameters
        df.at[ID, "Benchmark_name_length"] = len(benchmark_name)
        
%cd "/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track"

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/protostuff__protostuff/protostuff-protostuff-1.5.0/protostuff-benchmarks/src/main/java/io/protostuff/benchmarks
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/netty__netty/netty-netty-4.1.56.Final/microbench/src/main/java/io/netty/microbenchmark/common
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__hive/hive-release-3.1.3-rc0/itests/hive-jmh/src/main/java/org/apache/hive/benchmark/vectorization/operators
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__logging-log4j2/logging-log4j2-rel-2.14.0/log4j-perf/src/main/java/org/apache/logging/log4j/perf/jmh
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/RoaringBitmap__RoaringBitmap/RoaringBitmap-RoaringBitmap-0.9.5/jmh/src/jmh/java/org/roaringbitmap/iteration
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__hive/hive-release-3.1.3-rc0/itests/hive-

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/eclipse__rdf4j/rdf4j-3.5.0/testsuites/benchmark/src/main/java/org/eclipse/rdf4j/benchmark
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/prometheus__client_java/client_java-parent-0.9.0/benchmark/src/main/java/io/prometheus/benchmark
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__logging-log4j2/logging-log4j2-rel-2.14.0/log4j-perf/src/main/java/org/apache/logging/log4j/layout/template/json
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/JCTools__JCTools/JCTools-3.2.0/jctools-benchmarks/src/main/java/org/jctools/jmh/latency
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/zalando__logbook/logbook-2.4.1/logbook-jmh/src/main/java/org/zalando/logbook
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/r2dbc__r2dbc-h2/r2dbc-h2-0.8.4.RELEASE/src/jmh/java/io/r2dbc/h2
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challe

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/yellowstonegames__SquidLib/SquidLib-3.0.2/squidlib-performance/src/main/java/squidpony/performance
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/prometheus__client_java/client_java-parent-0.9.0/benchmark/src/main/java/io/prometheus/benchmark
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/crate__crate/crate-4.1.0/benchmarks/src/test/java/io/crate/execution/engine/collect/collectors
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/h2oai__h2o-3/h2o-3-jenkins-master-5303/h2o-core/src/jmh/java/water/util
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/openzipkin__zipkin/zipkin-2.22.1/benchmarks/src/main/java/zipkin2/codec
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__tinkerpop/tinkerpop-3.4.9/gremlin-tools/gremlin-benchmark/src/main/java/org/apache/tinkerpop/gremlin/process
/home/amirmahdi/Documents/Uni/Dr E

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/r2dbc__r2dbc-h2/r2dbc-h2-0.8.4.RELEASE/src/jmh/java/io/r2dbc/h2
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/imglib__imglib2/imglib2-imglib2-5.11.1/src/test/java/net/imglib2/loops
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/netty__netty/netty-netty-4.1.56.Final/microbench/src/main/java/io/netty/microbench/internal
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/raphw__byte-buddy/byte-buddy-byte-buddy-1.10.19/byte-buddy-benchmark/src/main/java/net/bytebuddy/benchmark
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/prestodb__presto/presto-0.245/presto-main/src/test/java/com/facebook/presto
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/prestodb__presto/presto-0.245/presto-main/src/test/java/com/facebook/presto/operator
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/prometheus__client_jav

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/HdrHistogram__HdrHistogram/HdrHistogram-HdrHistogram-2.1.12/HdrHistogram-benchmarks/src/main/java/bench
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__logging-log4j2/logging-log4j2-rel-2.14.0/log4j-perf/src/main/java/org/apache/logging/log4j/perf/jmh
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/JCTools__JCTools/JCTools-3.2.0/jctools-benchmarks/src/main/java/org/jctools/jmh/collections
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/ReactiveX__RxJava/RxJava-3.0.9/src/jmh/java/io/reactivex/rxjava3/core
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/zalando__logbook/logbook-2.4.1/logbook-jmh/src/main/java/org/zalando/logbook
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/h2oai__h2o-3/h2o-3-jenkins-master-5303/h2o-core/src/jmh/java/water
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects

...

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/netty__netty/netty-netty-4.1.56.Final/microbench/src/main/java/io/netty/microbenchmark/common
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/cantaloupe-project__cantaloupe/cantaloupe-4.1.7/src/test/java/edu/illinois/library/cantaloupe/perf/processor/codec
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/openzipkin__zipkin/zipkin-2.22.1/benchmarks/src/main/java/zipkin2/codec
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/yellowstonegames__SquidLib/SquidLib-3.0.2/squidlib-performance/src/main/java/squidpony/performance
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/zalando__logbook/logbook-2.4.1/logbook-jmh/src/main/java/org/zalando/logbook
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/openzipkin__zipkin/zipkin-2.22.1/benchmarks/src/main/java/zipkin2/codec
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Tra

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/cantaloupe-project__cantaloupe/cantaloupe-4.1.7/src/test/java/edu/illinois/library/cantaloupe/perf/processor/codec
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/cantaloupe-project__cantaloupe/cantaloupe-4.1.7/src/test/java/edu/illinois/library/cantaloupe/perf/processor/codec
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__kafka/kafka-2.7.0/jmh-benchmarks/src/main/java/org/apache/kafka/jmh/record
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__tinkerpop/tinkerpop-3.4.9/gremlin-tools/gremlin-benchmark/src/main/java/org/apache/tinkerpop/gremlin/process
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/eclipse__eclipse-collections/eclipse-collections-10.4.0/jmh-tests/src/main/java/org/eclipse/collections/impl/jmh/map
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/r2dbc__r2dbc-h2/r2dbc-h2-0.8.4.RELEASE/

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__hive/hive-release-3.1.3-rc0/itests/hive-jmh/src/main/java/org/apache/hive/benchmark/vectorization/operators
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/r2dbc__r2dbc-h2/r2dbc-h2-0.8.4.RELEASE/src/jmh/java/io/r2dbc/h2
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/prometheus__client_java/client_java-parent-0.9.0/benchmark/src/main/java/io/prometheus/benchmark
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__kafka/kafka-2.7.0/jmh-benchmarks/src/main/java/org/apache/kafka/jmh/record
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/cantaloupe-project__cantaloupe/cantaloupe-4.1.7/src/test/java/edu/illinois/library/cantaloupe/perf/processor/codec
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/prometheus__client_java/client_java-parent-0.9.0/benchmark/src/main/java/io/prometheus/benchmark
/home/amirmahd

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__logging-log4j2/logging-log4j2-rel-2.14.0/log4j-perf/src/main/java/org/apache/logging/log4j/perf/jmh
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/eclipse-vertx__vert.x/src/test/benchmarks/io/vertx/benchmarks
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/netty__netty/netty-netty-4.1.56.Final/microbench/src/main/java/io/netty/microbench/buffer
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/apache__tinkerpop/tinkerpop-3.4.9/gremlin-tools/gremlin-benchmark/src/main/java/org/apache/tinkerpop/gremlin/process
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/zalando__logbook/logbook-2.4.1/logbook-jmh/src/main/java/org/zalando/logbook
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/protostuff__protostuff/protostuff-protostuff-1.5.0/protostuff-benchmarks/src/main/java/io/protostuff/benchmarks
/home/amirmahdi/Docu

/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/JCTools__JCTools/JCTools-3.2.0/jctools-benchmarks/src/main/java/org/jctools/channels/spsc
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/raphw__byte-buddy/byte-buddy-byte-buddy-1.10.19/byte-buddy-benchmark/src/main/java/net/bytebuddy/benchmark
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/JCTools__JCTools/JCTools-3.2.0/jctools-benchmarks/src/main/java/org/jctools/jmh/latency
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/eclipse__jetty.project/jetty.project-jetty-11.0.0/tests/jetty-jmh/src/main/java/org/eclipse/jetty/util/jmh
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/openzipkin__zipkin/zipkin-2.22.1/benchmarks/src/main/java/zipkin2
/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Projects/yellowstonegames__SquidLib/SquidLib-3.0.2/squidlib-performance/src/main/java/squidpony/performance
/home/amirmahdi/Documents/Uni

In [57]:
pd.read_csv("file_not_found.csv", index_col=False)

,Unnamed: 0,ID
0,0,4c6311fab4ea38af744e5799f38a2026c4ad2821
1,1,e04e63feab301cf618e02fa0aae763072f27a12d
2,2,1aaf2ba664e922601a0746d5152d1c08ab716ef1
3,3,6159afaf3bd92e1557f33dc585ef869f01487023
4,4,f83503f9c7ba2869bb0b72dc5d04f4db8f91683e
5,5,81512f3d747a141a18c77ec587e231245d29f011
6,6,05eb19c1c24ce236a1c567f264ccaa3300c2d3bc
7,7,4e53734c4514441986f6c47dbf59c84eca3624af
8,8,eb70724186051efe7f9cbb88b38bbdf65dc72df7
9,9,1093b6accdbb1f15638fa63c4a770c725cda698d


In [67]:
pd.read_csv("srcml_error_files.csv", index_col=False)

,Unnamed: 0,ID
0,0,f3ec1a5e310c83ba8f0450e408c0808fdb0a18de
1,1,ecbd3009664d8378cc8fb96f7877bc44cf874cb4
2,2,ea2d44d88f29ffe10a80be76548c7a63ef05db30
3,3,e7d871dded65925f6bc12e1340dd9ad471ce5033
4,4,90b72a5fc9bf80f8bbb9c1eb5abd48bd345e290c
5,5,74c2f599f2f666c9b22adada3ce9a6c164653809
6,6,f4686be33a010143641425b309049560a39d66b1
7,7,96364903e728d952d53c580b8f22d25ab7de33b5
8,8,d0c0902a0a22a7eb07633034c660be01eb29cc9c
9,9,77e334b6d04bc8cdac5bd012a9a427f8d2b41443


In [21]:
hash_dict["a267443dadce1f7dbbbdf8c7ed067e4b5d3a20d5"]

'yellowstonegames__SquidLib#squidpony.performance.DataStructureBenchmark.insertIntegerSquidOS#NEED_TO_ENLARGE=1&SIZE=65536.json'

In [29]:
directory = "/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/classification"

for filename in filesname:
    f = os.path.join(directory, filename)
    
    if os.path.isfile(f):
        j_file = open(f)
        classes_data = json.load(j_file)         # Loading the benchmark's classes which is in json
        
        classes_df = pd.DataFrame(classes_data) 
        classification = 1 if classes_df["run"][0] == "steady state" else 0
        Number_of_no_steady_forks = 0
        for fork in forks:
            if fork == "no steady state":
                Number_of_no_steady_forks += 1
        
        ID = hashlib.sha1(str.encode(filename)).hexdigest()
        
        df.at[ID, "Number_of_no_steady_forks"] = Number_of_no_steady_forks
        df.at[ID, "class"] = classification
        

In [30]:
df.to_csv("/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/extracted_data.csv")

In [31]:
df = pd.read_csv("/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/extracted_data.csv", index_col=False).set_index("ID")

In [32]:
df.columns.tolist()

['Project_Name',
 'Bench_File_Name',
 'Bench_Method_Name',
 'itr_to_stds_avrg',
 'itr_to_stds_std',
 'Num_of_change_points_avrg',
 'STD',
 'AVRG',
 'Bench_File_Nloc',
 'Bench_Method_Nloc',
 'Bench_Method_NestingLevel',
 'Bench_Method_CCN',
 'Bench_File_CCN_Avrg',
 'Number_of_tryCatch',
 'Number_of_switchCase',
 'Number_of_systemCalls',
 'Synchronization',
 'Number_of_forLoops',
 'Number_of_whileLoops',
 'Number_of_packages',
 'Number_of_functionCalls',
 'Number_of_parameters',
 'Benchmark_name_length',
 'Number_of_no_steady_forks',
 'class']

In [33]:
df

,Project_Name,Bench_File_Name,Bench_Method_Name,itr_to_stds_avrg,itr_to_stds_std,Num_of_change_points_avrg,STD,AVRG,Bench_File_Nloc,Bench_Method_Nloc,...,Number_of_systemCalls,Synchronization,Number_of_forLoops,Number_of_whileLoops,Number_of_packages,Number_of_functionCalls,Number_of_parameters,Benchmark_name_length,Number_of_no_steady_forks,class
ID,,,,,,,,,,,,,,,,,,,,,
91f071f1f69bf48297887879cf5e5613552a29bc,protostuff__protostuff,RuntimeSchemaBenchmark.java,runtime_sparse_deserialize_10_int_field,1740.1,1021.197968,3.1,6.510561e-08,1.869715e-07,277.0,6.0,...,0,0.0,0,0,22,2,0.0,27.0,0.0,1.0
fb2459909444ecbdf72e47a11179359e56778dcd,netty__netty,AsciiStringBenchmark.java,hashCodeBenchBytesOld,196.6,326.932470,1.3,6.980091e-09,1.831741e-07,55.0,8.0,...,0,0.0,1,0,12,4,1.0,25.0,0.0,1.0
1c51969ca9fc7c1b5a7ac533bfa76899ac30ed88,apache__hive,VectorGroupByOperatorBench.java,testAggCount,363.5,396.638689,1.8,1.043777e-08,8.916153e-08,177.0,3.0,...,0,0.0,0,0,43,1,6.0,31.0,0.0,1.0
2b3a09d375526c8e9c7641660de34577306fb84e,apache__logging-log4j2,ThreadsafeDateFormatBenchmark.java,fastDateFormat,2256.7,982.812602,7.5,2.670392e-07,6.533268e-07,183.0,3.0,...,1,0.0,0,0,14,2,0.0,34.0,0.0,0.0
ca5d32819509b08c929db359c6f8dddad84e62f4,RoaringBitmap__RoaringBitmap,BatchIteratorBenchmark.java,batchIterate,1628.1,734.577491,3.5,1.180394e-04,1.245521e-03,54.0,11.0,...,0,0.0,1,1,3,3,4.0,27.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1a3b2d7b3a6ece03ec9db59ba9385a7199cace1b,apache__tinkerpop,GremlinGroovyScriptEngineBenchmark.java,testEvalLongCached,1124.9,1081.148783,2.7,1.093954e-02,5.990179e-04,79.0,3.0,...,0,0.0,0,0,18,1,0.0,39.0,0.0,1.0
0b0eb41a2e416dd93cf156632edab78e7227fab6,eclipse__jetty.project,StringReplaceBenchmark.java,testJavaStringReplaceReduce,1833.0,913.468883,5.3,1.698672e-07,2.915286e-07,85.0,4.0,...,0,0.0,0,0,15,2,2.0,27.0,0.0,1.0
8ac7776547858abdceb1e0b055d762870adcc23a,imglib__imglib2,SyncedPositionablesBenchmark.java,benchmark5,582.0,0.000000,3.0,1.060187e-02,3.083067e-02,64.0,4.0,...,0,0.0,0,0,16,12,0.0,33.0,0.0,1.0
